In [42]:
import pandas as pd
import numpy as np
from datetime import datetime as date


df_EConsumption = pd.read_csv('Retail_sales_of_electricity.csv', header = 4, sep=',')
df_NGConsumption = pd.read_excel('NG_CONS_SUM_A_EPG0_VGT_MMCF_M.xls', sheet_name=1, header=2)
dfHDD = pd.read_csv('MER_T01_10.csv', sep = ',')
dfCDD = pd.read_csv('MER_T01_11.csv', sep = ',')


# Data wrangling NG Consumption dataframe


df_NGConsumption.columns = df_NGConsumption.columns.map(lambda x: str.replace(x, ' (Including Vehicle Fuel) (MMcf)', '')) #modify header row
df_NGConsumption.columns = df_NGConsumption.columns.map(lambda x: str.replace(x, 'Natural Gas Delivered to Consumers in ', '')) #modify header row
df_NGConsumption = df_NGConsumption.melt(id_vars = ['Date'], var_name='State', value_name='Natural Gas Consumption (MMcf)') #melt dataframe into long shape


# Data wrangling EG Consumption dataframe

# print(df_EConsumption['description'].unique())
df_EConsumption = df_EConsumption[df_EConsumption['description'].str.contains('all sectors')] #filter by 'all sectors' (giving us total consumption by State by Date)
df_EConsumption['description'] = df_EConsumption['description'].apply(lambda x: str.replace(x, ' : all sectors', '')) #clean descriptions column


df_EConsumption = df_EConsumption.reset_index()
df_EConsumption_Dates = np.array(df_EConsumption.columns[4:])
df_EConsumption = df_EConsumption.melt(id_vars = ['description'], var_name="Date", value_vars = df_EConsumption_Dates, value_name='Electrical Consumption (million kWh)')
df_EConsumption['Date'] = df_EConsumption['Date'].apply(lambda x: date.strptime(x, '%b %Y'))

# print(df_EConsumption.head(10))
# print(df_NGConsumption.head(10))

print(dfCDD)



          MSN  YYYYMM  Value  Column_Order  \
0     ZWCDPC1  194913    572             1   
1     ZWCDPC1  195013    296             1   
2     ZWCDPC1  195113    326             1   
3     ZWCDPC1  195213    503             1   
4     ZWCDPC1  195313    372             1   
...       ...     ...    ...           ...   
6885  ZWCDPUS  202311     20            10   
6886  ZWCDPUS  202312     11            10   
6887  ZWCDPUS  202313   1476            10   
6888  ZWCDPUS  202401      9            10   
6889  ZWCDPUS  202402     13            10   

                             Description    Unit  
0       Cooling Degree-Days, New England  Number  
1       Cooling Degree-Days, New England  Number  
2       Cooling Degree-Days, New England  Number  
3       Cooling Degree-Days, New England  Number  
4       Cooling Degree-Days, New England  Number  
...                                  ...     ...  
6885  Cooling Degree-Days, United States  Number  
6886  Cooling Degree-Days, United State

In [43]:
# Example aggregation

import pandas as pd
import numpy as np
from datetime import datetime as date

# Convert date columns to datetime format
df_EConsumption['Date'] = pd.to_datetime(df_EConsumption['Date'])
df_NGConsumption['Date'] = pd.to_datetime(df_NGConsumption['Date'])

# Function to safely convert YYYYMM to datetime
def safe_convert_yyyymm(yyyymm):
    try:
        return pd.to_datetime(str(yyyymm) + '01', format='%Y%m%d')
    except ValueError:
        return pd.NaT

# Data wrangling for dfCDD
dfCDD['Date'] = dfCDD['YYYYMM'].apply(safe_convert_yyyymm)
dfCDD = dfCDD.drop(columns=['YYYYMM'])
dfCDD = dfCDD.dropna(subset=['Date'])

dfCDD = dfCDD.melt(id_vars=['Date'], var_name='Description', value_name='CDD')
dfCDD['State'] = dfCDD['Description'].apply(lambda x: x.split(', ')[1] if ', ' in x else x)
dfCDD = dfCDD.drop(columns=['Description'])

# Data wrangling for dfHDD
dfHDD['Date'] = dfHDD['YYYYMM'].apply(safe_convert_yyyymm)
dfHDD = dfHDD.drop(columns=['YYYYMM'])
dfHDD = dfHDD.dropna(subset=['Date'])

dfHDD = dfHDD.melt(id_vars=['Date'], var_name='Description', value_name='HDD')
dfHDD['State'] = dfHDD['Description'].apply(lambda x: x.split(', ')[1] if ', ' in x else x)
dfHDD = dfHDD.drop(columns=['Description'])

# Rename 'description' to 'State' for consistency in merging
df_EConsumption.rename(columns={'description': 'State'}, inplace=True)

# Ensure numerical columns are properly converted to numeric types
df_EConsumption['Electrical Consumption (million kWh)'] = pd.to_numeric(df_EConsumption['Electrical Consumption (million kWh)'], errors='coerce')
df_NGConsumption['Natural Gas Consumption (MMcf)'] = pd.to_numeric(df_NGConsumption['Natural Gas Consumption (MMcf)'], errors='coerce')
dfCDD['CDD'] = pd.to_numeric(dfCDD['CDD'], errors='coerce')
dfHDD['HDD'] = pd.to_numeric(dfHDD['HDD'], errors='coerce')

# Merge datasets on 'Date' and 'State'
df_merged = pd.merge(df_EConsumption, df_NGConsumption, on=['Date', 'State'], how='outer')
df_merged = pd.merge(df_merged, dfCDD, on=['Date', 'State'], how='outer')
df_merged = pd.merge(df_merged, dfHDD, on=['Date', 'State'], how='outer')

# Fill missing values
df_merged.fillna(0, inplace=True)

# Derive additional metrics
df_merged['Total Energy Consumption (kWh + NG)'] = df_merged['Electrical Consumption (million kWh)'] * 1e6 + df_merged['Natural Gas Consumption (MMcf)'] * 1e3
df_merged['Energy Consumption per Degree Day'] = df_merged['Total Energy Consumption (kWh + NG)'] / (df_merged['CDD'] + df_merged['HDD'])

# Basic analysis
summary = df_merged.groupby('State').agg({
    'Electrical Consumption (million kWh)': 'sum',
    'Natural Gas Consumption (MMcf)': 'sum',
    'Total Energy Consumption (kWh + NG)': 'sum',
    'Energy Consumption per Degree Day': 'mean'
}).reset_index()

print(summary.head())

# Save summary to CSV
summary.to_csv('Energy_Consumption_Summary.csv', index=False)b


        State  Electrical Consumption (million kWh)  \
0     Alabama                             2026370.0   
1      Alaska                              140541.0   
2     Arizona                             1740130.0   
3    Arkansas                             1078421.0   
4  California                             5915169.0   

   Natural Gas Consumption (MMcf)  Total Energy Consumption (kWh + NG)  \
0                      12300226.0                         2.038670e+12   
1                       2172391.0                         1.427134e+11   
2                       8048126.0                         1.748178e+12   
3                       6380526.0                         1.084802e+12   
4                      50686254.0                         5.965855e+12   

   Energy Consumption per Degree Day  
0                                NaN  
1                                NaN  
2                                NaN  
3                                NaN  
4                            

## Next Steps

### What We Need from the User:
1. **Date Range**: We'll ask for a start and end date to filter the data.
2. **Location**: The user will specify the state they’re interested in (e.g., ‘California’).
3. **Metrics**: They’ll choose the type of energy consumption they want to see - options include:
    - Electrical consumption
    - Natural gas consumption
    - Total energy consumption

### Steps:
1. **Ask for Inputs**: We'll prompt the user to enter the state, start date, end date, and the type of energy consumption. If they input incorrectly, we’ll give them a friendly message to try again.
2. **Filter the Data**: Use the inputs to get the relevant slice of our dataset.
3. **Plot the Data**: We'll create a plot using Matplotlib and save it as a PNG file.

### How It Works:
- **Input Prompts**: Clear and simple instructions for the user to follow:
    - State name (e.g., 'California')
    - Start date (YYYY-MM-DD)
    - End date (YYYY-MM-DD)
    - Metrics ('electrical', 'natural_gas', or 'total')

- **Output**: A neat plot showing the energy consumption based on their inputs.